In [1]:
import pandas as pd
import pickle
import numpy as np

In [134]:
with open("final_jongro.pickle","rb") as fr:
    jongro = pickle.load(fr)
print(jongro.head())

        serial       date  pm10 pm10_class msg_10  pm25 pm25_class msg_25  \
0  V10O1610252 2018-04-01  76.9          1      0  48.4          2      0   
1  V10O1610252 2018-04-02  79.3          1      0  34.7          1      0   
2  V10O1610252 2018-04-03  89.6          2      0  40.1          2      0   
3  V10O1610252 2018-04-04  19.2          0      0  17.5          1      0   
4  V10O1610252 2018-04-05  15.0          0      0  11.7          0      0   

  pm_class pm_class_info  
0        2          pm25  
1        1          pm10  
2        2          pm10  
3        1          pm25  
4        0          pm10  


In [135]:
jongstation = ['V10O1611255','V10O1611170','V10O1610545','V10O1611658','V10O1611639','V10O1611151','V10O1610252',
 'V10O1611634','V10O1611722','V10O1611173','V10O1611623','V10O1611251','V10O1611145','V10O1612113','V10O1611220',
 'V10O1611645','V10O1611750','V10O1611684','V10O1611698','V10O1610567','V10O1611887','V10O1610543',
 'V10O1610544','V10O1610546','V10O1612106','V10O1610542','V10O1611172','V10O1610540','V10O1611258','V10O1611289']



# date = pd.read_csv('./환경기상데이터/종로구/V10O1610252.csv', encoding = 'utf-8')['tm']
# jong = pd.DataFrame(date)
tm = pd.read_csv('./환경기상데이터/종로구/V10O1611170.csv', encoding = 'cp949')['tm']
tm = pd.DataFrame(tm)
# tm = tm.set_index(tm)
tm = tm.reset_index()
tm['tm'] = pd.to_datetime(tm.tm, format="%Y%m%d%H%M")
tm = tm.set_index(tm['tm'])
del tm['tm']

In [136]:
pm10 = tm
pm25 = tm

In [137]:
for j in jongstation:
    
    a = pd.read_csv('./환경기상데이터/종로구/'+j.strip()+'.csv', encoding = 'cp949')
    a['tm'] = pd.to_datetime(a.tm, format="%Y%m%d%H%M")
    temp =  pd.DataFrame(a[['pm10']])
    temp.columns = [j+'_pm10']
    temp = temp.set_index(a['tm'])
    pm10 =  pd.concat([pm10,temp], axis=1)
    temp =  pd.DataFrame(a[['pm25']])
    temp.columns = [j+'_pm25']
    temp = temp.set_index(a['tm'])
    pm25 =  pd.concat([pm25,temp], axis=1)
    print(j)
pm10 = pm10.replace(-999,np.NaN).replace(-9999,np.NaN).replace('NaN',np.NaN)
pm25 = pm25.replace(-999,np.NaN).replace(-9999,np.NaN).replace('NaN',np.NaN)

V10O1611255
V10O1611170
V10O1610545
V10O1611658
V10O1611639
V10O1611151
V10O1610252
V10O1611634
V10O1611722
V10O1611173
V10O1611623
V10O1611251
V10O1611145
V10O1612113
V10O1611220
V10O1611645
V10O1611750
V10O1611684
V10O1611698
V10O1610567
V10O1611887
V10O1610543
V10O1610544
V10O1610546
V10O1612106
V10O1610542
V10O1611172
V10O1610540
V10O1611258
V10O1611289


In [138]:
del pm10['index']
del pm25['index']

In [139]:
# 이상치 변환
pm25['V10O1611289_pm25'].loc['2018-08-29 07:38:00'] = round((pm25['V10O1611289_pm25']['2018-08-29 07:37:00']+pm25['V10O1611289_pm25']['2018-08-29 07:39:00'])/2)
pm25['V10O1611698_pm25'].loc['2018-04-03 19:38:00'] = round((pm25['V10O1611698_pm25'].loc['2018-04-03 19:37:00']+pm25['V10O1611698_pm25'].loc['2018-04-03 19:39:00'])/2)

In [140]:
# station 평균
pm10_mean = pm10.mean(axis=1)
pm25_mean = pm25.mean(axis=1)

In [141]:
# 일 평균
jong_day_pm10 = pm10_mean.groupby(tm.index.date).mean()
jong_day_pm25 = pm25_mean.groupby(tm.index.date).mean()

In [142]:
jong_day_pm10 = pd.DataFrame(jong_day_pm10)
jong_day_pm25 = pd.DataFrame(jong_day_pm25)
jong_day_pm10.columns = ['pm10_mean']
jong_day_pm25.columns = ['pm25_mean']

In [143]:
jong_day_pm10['pm10_class'] = 1
jong_day_pm10.loc[jong_day_pm10['pm10_mean']< 31.0 , 'pm10_class'] = 0                           # 좋음(0): 0 < pm10 < 30
jong_day_pm10.loc[(jong_day_pm10['pm10_mean']< 81.0) & (jong_day_pm10['pm10_mean'] >= 31.0), 'pm10_class'] = 1    # 보통(1) :31 < pm10 < 80
jong_day_pm10.loc[(jong_day_pm10['pm10_mean']< 151.0) & (jong_day_pm10['pm10_mean']>= 81.0) , 'pm10_class'] = 2   # 나쁨(2): 81 < pm10 < 150
jong_day_pm10.loc[jong_day_pm10['pm10_mean']>= 151.0 , 'pm10_class'] = 3                         # 매우나쁨(3): pm10 >= 151

In [144]:
jong_day_pm25['pm25_class'] = 1
jong_day_pm25.loc[jong_day_pm25['pm25_mean']< 16.0 , 'pm25_class'] = 0                           # 좋음(0): 0 < pm10 < 30
jong_day_pm25.loc[(jong_day_pm25['pm25_mean']< 36.0) & (jong_day_pm25['pm25_mean'] >= 16.0), 'pm25_class'] = 1    # 보통(1) :31 < pm10 < 80
jong_day_pm25.loc[(jong_day_pm25['pm25_mean']< 76.0) & (jong_day_pm25['pm25_mean']>= 36.0) , 'pm25_class'] = 2   # 나쁨(2): 81 < pm10 < 150
jong_day_pm25.loc[jong_day_pm25['pm25_mean']>= 76.0 , 'pm25_class'] = 3                         # 매우나쁨(3): pm10 >= 151

In [145]:
# jong_day_pm25.reset_index()
jong_totalmean = pd.concat([jong_day_pm10,jong_day_pm25], axis = 1)

In [146]:
jong_totalmean.loc[jong_totalmean['pm10_class']< jong_totalmean['pm25_class'] , 'pm_class'] = jong_totalmean['pm25_class']
jong_totalmean.loc[jong_totalmean['pm10_class']< jong_totalmean['pm25_class'] , 'pm_class_info'] ='pm25'

jong_totalmean.loc[jong_totalmean['pm10_class']>= jong_totalmean['pm25_class'] , 'pm_class'] = jong_totalmean['pm10_class']
jong_totalmean.loc[jong_totalmean['pm10_class']>= jong_totalmean['pm25_class'] , 'pm_class_info'] ='pm10'

jong_totalmean['pm10_class'] = jong_totalmean['pm10_class'].astype('int').astype('category')
jong_totalmean['pm25_class'] = jong_totalmean['pm25_class'].astype('int').astype('category')
jong_totalmean['pm_class'] = jong_totalmean['pm_class'].astype('int')
jong_totalmean['pm_class_info'] = jong_totalmean['pm_class_info'].astype('category')
jong_totalmean

,pm10_mean,pm10_class,pm25_mean,pm25_class,pm_class,pm_class_info
2018-04-01,63.716746,1,30.679210,1,1,pm10
2018-04-02,63.009583,1,24.875278,1,1,pm10
2018-04-03,61.819627,1,18.580544,1,1,pm10
2018-04-04,18.642072,0,9.441924,0,0,pm10
2018-04-05,11.689422,0,7.149783,0,0,pm10
2018-04-06,73.979994,1,18.425912,1,1,pm10
2018-04-07,44.598191,1,19.948118,1,1,pm10
2018-04-08,30.637063,0,14.931503,0,0,pm10
2018-04-09,42.743588,1,27.868171,1,1,pm10
2018-04-10,46.599632,1,22.260916,1,1,pm10


In [147]:
import pickle
with open("jong_totalmean","wb") as fr:
    pickle.dump(jong_totalmean,fr)

In [110]:
with open("final_nowon.pickle","rb") as fr:
    nowon = pickle.load(fr)
print(nowon.head())

        serial       date   pm10 pm10_class msg_10  pm25 pm25_class msg_25  \
0  V10O1610102 2018-05-07   95.6          2      0  43.5          2      0   
1  V10O1610102 2018-05-10  102.3          2      0  66.5          2      0   
2  V10O1610102 2018-05-11  158.5          3      0  93.7          3      1   
3  V10O1610102 2018-05-12   89.0          2      0  52.9          2      0   
4  V10O1610102 2018-05-13   54.3          1      0  43.6          2      0   

  pm_class pm_class_info  
0        2          pm10  
1        2          pm10  
2        3          pm10  
3        2          pm10  
4        2          pm25  


In [86]:
leng = [ (i,len(nowon.loc[nowon['serial']==i])) for i in list(set(nowon['serial']))]
leng

In [111]:
tm = nowon.loc[nowon['serial']=='V10O1611150']['date']
tm = pd.DataFrame(tm)
tm = tm.reset_index()
# tm['tm'] = pd.to_datetime(tm.tm, format="%Y%m%d%H%M")
tm = tm.set_index(tm['date'])
del tm['date']
pm10 = tm
pm25 = tm

In [112]:
for j in list(set(nowon['serial'])):
    
    a = nowon.loc[nowon['serial']==j]
#     a['tm'] = pd.to_datetime(a.tm, format="%Y%m%d%H%M")
    temp =  pd.DataFrame(a[['pm10']])
    temp.columns = [j+'_pm10']
    temp = temp.set_index(a['date'])
    pm10 =  pd.concat([pm10,temp], axis=1)
    temp =  pd.DataFrame(a[['pm25']])
    temp.columns = [j+'_pm25']
    temp = temp.set_index(a['date'])
    pm25 =  pd.concat([pm25,temp], axis=1)
    print(j)
pm10 = pm10.replace(-999,np.NaN).replace(-9999,np.NaN).replace('NaN',np.NaN)
pm25 = pm25.replace(-999,np.NaN).replace(-9999,np.NaN).replace('NaN',np.NaN)

V10O1610293
V10O1610376
V10O1610102
V10O1610630
V10O1611097
V10O1610642
V10O1610610
V10O1611104
V10O1611150
V10O1610629
V10O1610643
V10O1612126
V10O1611229
V10O1610616
V10O1610356
V10O1610200
V10O1611102
V10O1611100
V10O1610297
V10O1610351
V10O1610312
V10O1611652


In [124]:
del pm10['index']
del pm25['index']

In [125]:
nowon_day_pm10 = pd.DataFrame(pm10.mean(axis=1))
nowon_day_pm25 = pd.DataFrame(pm25.mean(axis=1))
nowon_day_pm10.columns = ['pm10_mean']
nowon_day_pm25.columns = ['pm25_mean']

In [126]:
nowon_day_pm10['pm10_class'] = 1
nowon_day_pm10.loc[nowon_day_pm10['pm10_mean']< 31.0 , 'pm10_class'] = 0                           # 좋음(0): 0 < pm10 < 30
nowon_day_pm10.loc[(nowon_day_pm10['pm10_mean']< 81.0) & (nowon_day_pm10['pm10_mean'] >= 31.0), 'pm10_class'] = 1    # 보통(1) :31 < pm10 < 80
nowon_day_pm10.loc[(nowon_day_pm10['pm10_mean']< 151.0) & (nowon_day_pm10['pm10_mean']>= 81.0) , 'pm10_class'] = 2   # 나쁨(2): 81 < pm10 < 150
nowon_day_pm10.loc[nowon_day_pm10['pm10_mean']>= 151.0 , 'pm10_class'] = 3                         # 매우나쁨(3): pm10 >= 151

In [127]:
nowon_day_pm25['pm25_class'] = 1
nowon_day_pm25.loc[nowon_day_pm25['pm25_mean']< 16.0 , 'pm25_class'] = 0                           # 좋음(0): 0 < pm10 < 30
nowon_day_pm25.loc[(nowon_day_pm25['pm25_mean']< 36.0) & (nowon_day_pm25['pm25_mean'] >= 16.0), 'pm25_class'] = 1    # 보통(1) :31 < pm10 < 80
nowon_day_pm25.loc[(nowon_day_pm25['pm25_mean']< 76.0) & (nowon_day_pm25['pm25_mean']>= 36.0) , 'pm25_class'] = 2   # 나쁨(2): 81 < pm10 < 150
nowon_day_pm25.loc[nowon_day_pm25['pm25_mean']>= 76.0 , 'pm25_class'] = 3                         # 매우나쁨(3): pm10 >= 151

In [128]:
nowon_totalmean = pd.concat([nowon_day_pm10,nowon_day_pm25], axis = 1)

In [130]:
nowon_totalmean.head()

,pm10_mean,pm10_class,pm25_mean,pm25_class
date,,,,
2018-04-01,84.090,2,50.175,2
2018-04-02,81.275,2,36.230,2
2018-04-03,93.490,2,41.215,2
2018-04-04,14.350,0,7.890,0
2018-04-05,33.925,1,8.250,0


In [153]:
nowon_totalmean.loc[nowon_totalmean['pm10_class']< nowon_totalmean['pm25_class'] , 'pm_class'] = nowon_totalmean['pm25_class']
nowon_totalmean.loc[nowon_totalmean['pm10_class']< nowon_totalmean['pm25_class'] , 'pm_class_info'] ='pm25'

nowon_totalmean.loc[nowon_totalmean['pm10_class']>= nowon_totalmean['pm25_class'] , 'pm_class'] = nowon_totalmean['pm10_class']
nowon_totalmean.loc[nowon_totalmean['pm10_class']>= nowon_totalmean['pm25_class'] , 'pm_class_info'] ='pm10'

nowon_totalmean['pm10_class'] = nowon_totalmean['pm10_class'].astype('int').astype('category')
nowon_totalmean['pm25_class'] = nowon_totalmean['pm25_class'].astype('int').astype('category')
nowon_totalmean['pm_class'] = nowon_totalmean['pm_class'].astype('int')
nowon_totalmean['pm_class_info'] = nowon_totalmean['pm_class_info'].astype('category')
nowon_totalmean

,pm10_mean,pm10_class,pm25_mean,pm25_class,pm_class,pm_class_info
date,,,,,,
2018-04-01,84.090000,2,50.175000,2,2,pm10
2018-04-02,81.275000,2,36.230000,2,2,pm10
2018-04-03,93.490000,2,41.215000,2,2,pm10
2018-04-04,14.350000,0,7.890000,0,0,pm10
2018-04-05,33.925000,1,8.250000,0,1,pm10
2018-04-06,218.721053,3,86.163158,3,3,pm10
2018-04-07,64.031579,1,35.394737,1,1,pm10
2018-04-08,43.021053,1,22.642105,1,1,pm10
2018-04-09,65.330000,1,43.040000,2,2,pm25


In [154]:
with open("nowon_totalmean","wb") as fr:
    pickle.dump(nowon_totalmean,fr)